Create Table in Redshift using below script.
```sql
CREATE TABLE order_revenue (
    order_date DATE,
    order_status VARCHAR,
    order_item_product_id INT,
    order_revenue FLOAT
);

ALTER TABLE order_revenue
    OWNER TO retail_user;
```

In [16]:
import boto3

In [17]:
rsd_client = boto3.client('redshift-data')

In [18]:
rs_copy_command = '''
    COPY order_revenue FROM 's3://airetail/retail_db_json/order_revenue' 
    IAM_ROLE 'arn:aws:iam::269066542444:role/AIRedshiftRetailSpectrum'
    JSON AS 'auto'
'''

In [19]:
waiter_name = 'DataAPIExecution'

delay=2
max_attempts=3

#Configure the waiter settings
waiter_config = {
  'version': 2,
  'waiters': {
    'DataAPIExecution': {
      'operation': 'DescribeStatement',
      'delay': delay,
      'maxAttempts': max_attempts,
      'acceptors': [
        {
          "matcher": "path",
          "expected": "FINISHED",
          "argument": "Status",
          "state": "success"
        },
        {
          "matcher": "pathAny",
          "expected": ["PICKED","STARTED","SUBMITTED"],
          "argument": "Status",
          "state": "retry"
        },
        {
          "matcher": "pathAny",
          "expected": ["FAILED","ABORTED"],
          "argument": "Status",
          "state": "failure"
        }
      ],
    },
  },
}

In [20]:
from botocore.exceptions import WaiterError
from botocore.waiter import WaiterModel
from botocore.waiter import create_waiter_with_client

In [21]:
waiter_model = WaiterModel(waiter_config)

In [22]:
custom_waiter = create_waiter_with_client(waiter_name, waiter_model, rsd_client)

In [26]:
rs_copy_command_id = rsd_client.execute_statement(
    WorkgroupName='retail-wg',
    Database='retail_db',
    Sql=rs_copy_command
)['Id']

try:
    custom_waiter.wait(Id=rs_copy_command_id)    
except WaiterError as e:
    print (e)


In [27]:
rsd_client.describe_statement(Id=rs_copy_command_id)

{'CreatedAt': datetime.datetime(2022, 7, 18, 18, 28, 21, 403000, tzinfo=tzlocal()),
 'Duration': 1665670121,
 'HasResultSet': False,
 'Id': '8693c9b8-7a13-40df-98d8-8adcf2a57411',
 'QueryString': "\n    COPY order_revenue FROM 's3://airetail/retail_db_json/order_revenue' \n    IAM_ROLE ''\n    JSON AS 'auto'\n",
 'RedshiftPid': 1073837843,
 'RedshiftQueryId': 34908,
 'ResultRows': 0,
 'ResultSize': 0,
 'Status': 'FINISHED',
 'UpdatedAt': datetime.datetime(2022, 7, 18, 18, 28, 23, 520000, tzinfo=tzlocal()),
 'WorkgroupName': 'retail-wg',
 'ResponseMetadata': {'RequestId': '77467c38-9928-47e3-8224-a23b56c31f84',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '77467c38-9928-47e3-8224-a23b56c31f84',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '403',
   'date': 'Mon, 18 Jul 2022 12:58:27 GMT'},
  'RetryAttempts': 0}}

In [10]:
order_revenue_query = '''SELECT count(*) FROM order_revenue'''

In [11]:
order_revenue_query_id = rsd_client.execute_statement(
    WorkgroupName='retail-wg',
    Database='retail_db',
    Sql=order_revenue_query
)['Id']

In [12]:
rsd_client.get_statement_result(Id=order_revenue_query_id)['Records']

[[{'longValue': 38385}]]